In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import torch 
import torch.optim as optim
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset,Dataset
from sklearn.model_selection import KFold
import sys
sys.path.append('D:\Desktop\Вся прога на питоне\Задача по курсовой')
from functions import ONP_Adagrad,ONP,to_exel

<>:12: SyntaxWarning: invalid escape sequence '\D'
<>:12: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Роман\AppData\Local\Temp\ipykernel_9504\368418371.py:12: SyntaxWarning: invalid escape sequence '\D'
  sys.path.append('D:\Desktop\Вся прога на питоне\Задача по курсовой')


In [2]:
train = pd.read_csv('D:/Desktop/Вся прога на питоне/Задача по курсовой/mtsgrvmgn_trn.csv')
test = pd.read_csv('D:/Desktop/Вся прога на питоне/Задача по курсовой/mtsgrvmgn_tst.csv')
valid = pd.read_csv('D:/Desktop/Вся прога на питоне/Задача по курсовой/mtsgrvmgn_vld.csv')

In [3]:
y_train = train[['H1_8','H2_8','H3_8']]
y_valid = valid[['H1_8','H2_8','H3_8']]
y_test = test[['H1_8','H2_8','H3_8']]
X_test = test.drop(['H1_8', 'H2_8', 'H3_8'], axis=1)
common_columns = X_test.columns.intersection(train.columns)
X_train = train[common_columns]
X_test = X_test[common_columns]
X_valid = valid[common_columns]

to_exel(file_name='All_3.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_train,y_train=y_train,
        X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
        batch_size=64,input_dim=X_train.shape[1],output_dim=3,
        learning_rate=0.001,num_epochs=50,patience=10)

Early stopping triggered at epoch 28
Early stopping triggered at epoch 23
Early stopping triggered at epoch 19


In [5]:
np.max(y_train['H3_8'])

2.28

In [6]:
np.min(y_train['H1_8'])

1.02

In [8]:
np.max(y_train['H3_8'])-np.min(y_train['H3_8'])

0.07999999999999963

In [ ]:
with pd.ExcelWriter('output_Adam_All.xlsx', engine='openpyxl') as writer:
    for i in range(3):
                TE,CV,CVE = ONP(X_train=X_train,y_train=y_train,X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
                batch_size=64,input_dim=X_train.shape[1],output_dim=3,learning_rate=0.001,num_epochs=50,patience=10,graph=False)
                if i == 0:
                    TE.to_excel(writer, sheet_name='TE', index=False, startrow=0, startcol=i * 4)
                else:
                    TE.to_excel(writer, sheet_name='TE', index=False, startrow=0, startcol=i * 4 + 1)
                # Записываем CV
                if i == 0:
                    CV.to_excel(writer, sheet_name='CV', index=False, startrow=0, startcol=i * 4)
                else:
                    CV.to_excel(writer, sheet_name='CV', index=False, startrow=0, startcol=i * 4 + 1)
                # Записываем CVE
                if i == 0:
                    CVE.to_excel(writer, sheet_name='CVE', index=False, startrow=0, startcol=i * 4)
                else:
                    CVE.to_excel(writer, sheet_name='CVE', index=False, startrow=0, startcol=i * 4 + 1)
    

Early stopping triggered at epoch 30
Early stopping triggered at epoch 45


In [ ]:
Lr = [0.001,0.01,0.1]
for lr in Lr:
    file_name = f"lr{lr},3A.xlsx"
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        for i in range(2):
            TE,CV,CVE = ONP(X_train=X_train,y_train=y_train,X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
            batch_size=64,input_dim=X_train.shape[1],output_dim=3,learning_rate=lr,num_epochs=50,patience=10,graph=False)
            if i == 0:
                TE.to_excel(writer, sheet_name='TE', index=False, startrow=0, startcol=i * 3)
            else:
                TE.to_excel(writer, sheet_name='TE', index=False, startrow=0, startcol=i * 3 + 1)
            # Записываем CV
            if i == 0:
                CV.to_excel(writer, sheet_name='CV', index=False, startrow=0, startcol=i * 3)
            else:
                CV.to_excel(writer, sheet_name='CV', index=False, startrow=0, startcol=i * 3 + 1)
            # Записываем CVE
            if i == 0:
                CVE.to_excel(writer, sheet_name='CVE', index=False, startrow=0, startcol=i * 3)
            else:
                CVE.to_excel(writer, sheet_name='CVE', index=False, startrow=0, startcol=i * 3 + 1)
    

In [5]:
k = 0
corr_ = 0
corr_h=[]
corr_s = 0

idx = [0,0,0]
for i in X_train.columns:
    corr_h=[]
    corr_h.append( abs(pearsonr(X_train[i], y_train['H1_8'])[0]))
    corr_h.append(abs(pearsonr(X_train[i], y_train['H2_8'])[0]))
    corr_h.append (abs( pearsonr(X_train[i], y_train['H3_8'])[0]))
    corr_s = np.mean(corr_h)
    if corr_<abs(corr_s): 
        corr_=corr_s
        idx[0] = i
print(idx,corr_)

['IMYX10_16', 0, 0] 0.44012605833442536


In [5]:
X_t_1 = X_train.copy()
X_v_1 = X_valid.copy()
X_te_1 = X_test.copy()
for i in X_train.columns:
    res = pearsonr(X_train[i], X_train['IMYX10_16'])
    if abs(res[0])>0.9 or abs(res[0])<0.15:
        X_t_1 = X_t_1.drop(i,axis = 1)
        X_v_1 = X_v_1.drop(i,axis = 1)
        X_te_1 = X_te_1.drop(i,axis = 1)
print(X_t_1.shape)

to_exel(file_name='filtrP_3.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_t_1,y_train=y_train,
        X_valid=X_v_1,y_valid=y_valid,X_test=X_te_1,y_test=y_test,
        batch_size=64,input_dim=X_t_1.shape[1],output_dim=3,
        learning_rate=0.001,num_epochs=50,patience=10)

(21000, 745)
Early stopping triggered at epoch 49
Early stopping triggered at epoch 40


In [6]:
# H1 не фильтрованная
y_train = train[['H1_8']]
y_valid = valid[['H1_8']]
y_test = test[['H1_8']]
X_test = test.drop(['H1_8'], axis=1)
common_columns = X_test.columns.intersection(train.columns)
X_train = train[common_columns]
X_test = X_test[common_columns]
X_valid = valid[common_columns]
to_exel(file_name='All_H1.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_train,y_train=y_train,
        X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
        batch_size=64,input_dim=X_train.shape[1],output_dim=1,
        learning_rate=0.001,num_epochs=50,patience=10)

Early stopping triggered at epoch 26
Early stopping triggered at epoch 19
Early stopping triggered at epoch 34


In [7]:
#Для H2 нефильтрованного 
y_train = train[['H2_8']]
y_valid = valid[['H2_8']]
y_test = test[['H2_8']]
X_test = test.drop(['H2_8'], axis=1)
common_columns = X_test.columns.intersection(train.columns)
X_train = train[common_columns]
X_test = X_test[common_columns]
X_valid = valid[common_columns]
to_exel(file_name='All_H2.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_train,y_train=y_train,
        X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
        batch_size=64,input_dim=X_train.shape[1],output_dim=1,
        learning_rate=0.001,num_epochs=50,patience=10)

Early stopping triggered at epoch 34
Early stopping triggered at epoch 44
Early stopping triggered at epoch 37


In [4]:
# Для H3 нефильтрованного 
y_train = train[['H3_8']]
y_valid = valid[['H3_8']]
y_test = test[['H3_8']]
X_test = test.drop(['H3_8'], axis=1)
common_columns = X_test.columns.intersection(train.columns)
X_train = train[common_columns]
X_test = X_test[common_columns]
X_valid = valid[common_columns]
to_exel(file_name='All_H3.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_train,y_train=y_train,
        X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
        batch_size=64,input_dim=X_train.shape[1],output_dim=1,
        learning_rate=0.001,num_epochs=50,patience=10)

Early stopping triggered at epoch 50
Early stopping triggered at epoch 48


In [ ]:
k = 0
corr_ = 0
corr_h=[]
corr_s = 0

idx = [0,0,0]
for i in X_train.columns:
    corr_h=[]
    corr_h.append (abs( pearsonr(X_train[i], y_train['H3_8'])[0]))
    if corr_<abs(corr_s): 
        corr_=corr_s
        idx[0] = i
print(idx,corr_)

['H1_9', 0, 0] 0.3640140380997824


In [9]:
# Для H3 фильтрованного по Пирсону
X_t_1 = X_train.copy()
X_v_1 = X_valid.copy()
X_te_1 = X_test.copy()
for i in X_train.columns:
    res = pearsonr(X_train[i], X_train['H1_9'])
    if abs(res[0])>0.9 or abs(res[0])<0.15:
        X_t_1 = X_t_1.drop(i,axis = 1)
        X_v_1 = X_v_1.drop(i,axis = 1)
        X_te_1 = X_te_1.drop(i,axis = 1)
print(X_t_1.shape)
X_train = X_t_1.copy()
X_valid = X_v_1.copy()
X_test = X_te_1.copy()
to_exel(file_name='filtrP_H3.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_train,y_train=y_train,
        X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
        batch_size=64,input_dim=X_train.shape[1],output_dim=1,
        learning_rate=0.001,num_epochs=50,patience=10)

(21000, 787)


In [10]:
# Для H1 фильтрованного по Пирсону
y_train = train[['H1_8']]
y_valid = valid[['H1_8']]
y_test = test[['H1_8']]
X_test = test.drop(['H1_8'], axis=1)
common_columns = X_test.columns.intersection(train.columns)
X_train = train[common_columns]
X_test = X_test[common_columns]
X_valid = valid[common_columns]

In [11]:
k = 0
corr_ = 0
corr_h=[]
corr_s = 0

idx = [0,0,0]
for i in X_train.columns:
    corr_h=[]
    corr_h.append (abs( pearsonr(X_train[i], y_train['H1_8'])[0]))
    corr_s = abs( pearsonr(X_train[i], y_train['H1_8'])[0])
    if corr_<abs(corr_s): 
        corr_=corr_s
        idx[0] = i
print(idx,corr_)

['IMYX4_16', 0, 0] 0.9469079498595657


In [12]:
X_t_1 = X_train.copy()
X_v_1 = X_valid.copy()
X_te_1 = X_test.copy()
for i in X_train.columns:
    res = pearsonr(X_train[i], X_train['IMYX4_16'])
    if abs(res[0])>0.9 or abs(res[0])<0.15:
        X_t_1 = X_t_1.drop(i,axis = 1)
        X_v_1 = X_v_1.drop(i,axis = 1)
        X_te_1 = X_te_1.drop(i,axis = 1)
print(X_t_1.shape)
X_train = X_t_1.copy()
X_valid = X_v_1.copy()
X_test = X_te_1.copy()
to_exel(file_name='filtrP_H1.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_train,y_train=y_train,
        X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
        batch_size=64,input_dim=X_train.shape[1],output_dim=1,
        learning_rate=0.001,num_epochs=50,patience=10)

(21000, 915)
Early stopping triggered at epoch 41


In [13]:
# Для H2 фильтрованного по Пирсону
y_train = train[['H2_8']]
y_valid = valid[['H2_8']]
y_test = test[['H2_8']]
X_test = test.drop(['H2_8'], axis=1)
common_columns = X_test.columns.intersection(train.columns)
X_train = train[common_columns]
X_test = X_test[common_columns]
X_valid = valid[common_columns]

In [14]:
k = 0
corr_ = 0
corr_h=[]
corr_s = 0

idx = [0,0,0]
for i in X_train.columns:
    corr_h=[]
    corr_h.append (abs( pearsonr(X_train[i], y_train['H2_8'])[0]))
    corr_s = abs( pearsonr(X_train[i], y_train['H2_8'])[0])
    if corr_<abs(corr_s): 
        corr_=corr_s
        idx[0] = i
print(idx,corr_)

['REYX5_16', 0, 0] 0.5909479154084429


In [15]:
X_t_1 = X_train.copy()
X_v_1 = X_valid.copy()
X_te_1 = X_test.copy()
for i in X_train.columns:
    res = pearsonr(X_train[i], X_train['REYX5_16'])
    if abs(res[0])>0.9 or abs(res[0])<0.15:
        X_t_1 = X_t_1.drop(i,axis = 1)
        X_v_1 = X_v_1.drop(i,axis = 1)
        X_te_1 = X_te_1.drop(i,axis = 1)
print(X_t_1.shape)
X_train = X_t_1.copy()
X_valid = X_v_1.copy()
X_test = X_te_1.copy()
to_exel(file_name='filtrP_H2.xlsx',
        n_coloumns=4,n_iter=3,X_train=X_train,y_train=y_train,
        X_valid=X_valid,y_valid=y_valid,X_test=X_test,y_test=y_test,
        batch_size=64,input_dim=X_train.shape[1],output_dim=1,
        learning_rate=0.001,num_epochs=50,patience=10)

(21000, 810)
